In [19]:
# PACOTES

import pandas as pd
import requests
import os
import json
import re
from dotenv import load_dotenv

In [20]:
# PARÂMETROS

load_dotenv(override=True)
access_token = os.getenv("access_token")
headers = {'X-GitHub-Api-Version': '2022-11-28','Authorization': 'Bearer' + access_token}
owner = 'amzn'
url = 'https://api.github.com'
url_formatted = f'{url}/users/{owner}/repos'
save_data = r'..\data\repos_data.csv'

In [21]:
# REQUISIÇÃO

response = requests.get(url_formatted, headers=headers)

if response.status_code:
    print('success')
else:
    print('error')

success


In [22]:
# PAGINAÇÃO

if response.status_code == 200:
    link_header = response.headers.get('Link')
    if link_header:
        links = link_header.split(',')
        last_url = None
        for link in links:
            if 'rel="last"' in link:
                last_url = link.split(';')[0].strip('<>')
                break

if last_url:
    match = re.search(r'page=(\d+)', last_url)
    if match:
        last_page = match.group(1)
        last_page = int(last_page)

print(last_page)

6


In [23]:
# EXTRAÇÃO GERAL

repos_list = []

for page_num in range(1,last_page+1):
    try:
        url_page = f'{url_formatted}?page={page_num}'
        response = requests.get(url_page, headers=headers)
        repos_list.append(response.json())
    except:
        repos_list.append(None)

In [24]:
# EXTRAÇÃO DAS INFORMAÇÕES

repos_name = []
repos_description = []
repos_language = []
repos_created_at = []
repos_updated_at = []

for page in repos_list:
    for repo in page:        
        repos_name.append(repo['name'])
        repos_description.append(repo['description'])
        repos_language.append(repo['language'])
        repos_created_at.append(repo['created_at'])
        repos_updated_at.append(repo['updated_at'])

In [25]:
# CRIANDO DATAFRAME

df_repos = pd.DataFrame()
df_repos['repo_name'] = repos_name
df_repos['repo_description'] = repos_description
df_repos['repo_language'] = repos_language
df_repos['repo_created_at'] = repos_created_at
df_repos['repos_updated_at'] = repos_updated_at

In [26]:
# CHECANDO DADOS

df_repos.head()

,repo_name,repo_description,repo_language,repo_created_at,repos_updated_at
0,.github,None,None,2019-02-18T18:59:22Z,2024-10-07T21:33:10Z
1,ads-advanced-tools-docs,Code samples and supplements for the Amazon Ad...,Jupyter Notebook,2022-05-16T19:49:03Z,2024-10-09T07:27:26Z
2,ads-pao-amznjs-gtm-template,None,Smarty,2023-08-02T23:41:56Z,2024-09-05T08:32:25Z
3,alexa-coho,Sample code for building skill adapters for Al...,JavaScript,2015-08-20T20:42:31Z,2024-07-11T01:03:40Z
4,alexa-skills-kit-js,SDK and example code for building voice-enabl...,None,2015-07-21T21:46:48Z,2024-08-04T00:31:19Z


In [27]:
# SALVANDO ARQUIVO

df_repos.to_csv(save_data, index=False)